<a href="https://colab.research.google.com/github/jjefferson2006/DSWP/blob/master/desafio_churn_ultima1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, accuracy_score, confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from pycaret.classification import *

In [4]:
url = 'https://raw.githubusercontent.com/jjefferson2006/DSWP_JJ_altera/main/train.csv'
df_churn_train = pd.read_csv(url)
df_churn_train

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,6684,Male,0,No,No,43.0,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,Electronic check,44.15,1931.3,0
5630,604,Male,0,No,No,NaN,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,No,Electronic check,108.65,4903.2,0
5631,4803,Female,0,No,No,28.0,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,Bank transfer (automatic),54.65,1517.5,0
5632,6317,Male,0,Yes,Yes,63.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.25,1573.05,0


In [5]:
#Normalizar as colunas
df_churn_train.columns = [col.lower() for col in df_churn_train.columns]
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [6]:
df_churn_train.dtypes

id                    int64
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure              float64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                 int64
dtype: object

In [7]:
df_churn_train.isna().sum()

id                    0
gender                0
seniorcitizen         0
partner               0
dependents          218
tenure              461
phoneservice          0
multiplelines         0
internetservice       0
onlinesecurity        0
onlinebackup          0
deviceprotection      0
techsupport           0
streamingtv           0
streamingmovies       0
contract              0
paperlessbilling      0
paymentmethod        99
monthlycharges        0
totalcharges          0
churn                 0
dtype: int64

In [8]:
df_churn_train.shape

(5634, 21)

###pay

In [9]:
#contar a variavel 
df_churn_train['paymentmethod'].value_counts()

Electronic check             1887
Mailed check                 1272
Bank transfer (automatic)    1212
Credit card (automatic)      1164
Name: paymentmethod, dtype: int64

In [10]:
df_churn_train['paymentmethod'].isna().sum()

99

In [11]:
df_churn_train['paymentmethod'].fillna('Mailed check', inplace=True)

###trat dep

In [12]:
df_churn_train['dependents'].isna().sum()

218

In [13]:
#contar a variavel 
df_churn_train['dependents'].value_counts()

No     3793
Yes    1623
Name: dependents, dtype: int64

In [14]:
#criterio = moda
df_churn_train['dependents'].fillna('Yes', inplace=True)

###tratar as numeric

In [15]:
df_churn_train['totalcharges'] = df_churn_train['totalcharges'].str.replace(',', '.')
df_churn_train['totalcharges'] = pd.to_numeric(df_churn_train['totalcharges'], errors= 'coerce')



In [16]:
df_churn_train['monthlycharges'].isna().sum()

0

In [17]:
df_churn_train['totalcharges'].sum()

12790788.7

In [18]:
df_churn_train['tenure'].isna().sum()

461

In [19]:
df_churn_train['tenure_esti']=round(df_churn_train['totalcharges']/df_churn_train['monthlycharges'],2)

In [20]:
df_churn_train['tenure_rou']=round(df_churn_train['tenure_esti'])

In [21]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,tenure_esti,tenure_rou
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Mailed check,45.05,2560.10,0,56.83,57.0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0,16.93,17.0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0,59.93,60.0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1,34.79,35.0
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1,27.91,28.0


In [22]:
l_tot_ten = list(df_churn_train[df_churn_train['tenure'].isna()].index)
l_tot_ten

[1,
 11,
 34,
 45,
 55,
 92,
 94,
 109,
 121,
 133,
 185,
 190,
 193,
 200,
 221,
 223,
 231,
 234,
 247,
 251,
 268,
 288,
 298,
 323,
 332,
 346,
 353,
 368,
 372,
 381,
 390,
 398,
 408,
 409,
 436,
 453,
 463,
 464,
 465,
 469,
 472,
 474,
 483,
 485,
 486,
 508,
 520,
 544,
 559,
 560,
 572,
 596,
 602,
 620,
 632,
 640,
 682,
 700,
 707,
 773,
 787,
 791,
 809,
 811,
 830,
 832,
 851,
 854,
 880,
 886,
 923,
 925,
 950,
 956,
 958,
 968,
 988,
 992,
 1003,
 1049,
 1070,
 1090,
 1098,
 1105,
 1114,
 1115,
 1127,
 1135,
 1137,
 1143,
 1159,
 1169,
 1173,
 1192,
 1196,
 1206,
 1208,
 1214,
 1233,
 1238,
 1254,
 1261,
 1265,
 1266,
 1278,
 1279,
 1286,
 1336,
 1373,
 1405,
 1406,
 1414,
 1420,
 1440,
 1449,
 1475,
 1476,
 1490,
 1501,
 1503,
 1516,
 1526,
 1563,
 1584,
 1587,
 1591,
 1614,
 1637,
 1655,
 1681,
 1691,
 1695,
 1696,
 1703,
 1716,
 1718,
 1725,
 1734,
 1742,
 1750,
 1753,
 1770,
 1778,
 1789,
 1793,
 1802,
 1813,
 1817,
 1830,
 1845,
 1872,
 1887,
 1896,
 1906,
 1922,
 

In [23]:
for i in l_tot_ten:
    df_churn_train['tenure'][i] = df_churn_train['tenure_rou'][i]


In [24]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,tenure_esti,tenure_rou
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Mailed check,45.05,2560.10,0,56.83,57.0
1,6731,Male,0,Yes,Yes,17.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0,16.93,17.0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0,59.93,60.0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1,34.79,35.0
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1,27.91,28.0


In [25]:
df_churn_train.drop(columns=['tenure_esti','tenure_rou'], inplace=True)

In [26]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Mailed check,45.05,2560.10,0
1,6731,Male,0,Yes,Yes,17.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1


###trata categor

In [27]:
df_churn_train.gender.replace(['Female', 'Male'], [1, 0], inplace=True)
df_churn_train.partner.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_train.phoneservice.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_train.paperlessbilling.replace(['Yes', 'No'], [1, 0], inplace=True)

In [28]:
df_churn_train.internetservice.replace(['Fiber optic', 'DSL', 'No'], [1,2,0], inplace=True)
df_churn_train.paymentmethod.replace(['Electronic check', 'Bank transfer (automatic)', 'Mailed check', 'Credit card (automatic)'], [1,2,3,4], inplace=True)
df_churn_train.contract.replace(['Two year', 'One year', 'Month-to-month'], [0,1,2], inplace=True)
df_churn_train.multiplelines.replace(['No', 'Yes', 'No phone service'], [0,1,3], inplace=True)
df_churn_train.onlinesecurity.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 
df_churn_train.onlinebackup.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.deviceprotection.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.techsupport.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.streamingtv.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_train.streamingmovies.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 

In [29]:
df_churn_train.dependents.replace(['Yes', 'No'], [1, 0], inplace=True)

In [30]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,1,0,0,0,56.0,0,3,2,1,1,1,1,0,0,0,0,3,45.05,2560.10,0
1,6731,0,0,1,1,17.0,1,0,0,2,2,2,2,2,2,1,0,3,19.65,332.65,0
2,6479,1,0,1,0,60.0,1,1,1,0,1,1,1,1,1,0,1,4,110.80,6640.70,0
3,6861,1,0,0,0,37.0,1,1,1,0,1,1,0,1,1,2,1,2,101.90,3545.35,1
4,3266,0,0,1,1,29.0,1,0,1,0,0,0,0,0,0,2,1,1,70.75,1974.80,1


In [31]:
df_churn_train.isna().sum()

id                  0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        8
churn               0
dtype: int64

In [32]:
l_index_charg = list(df_churn_train[df_churn_train['totalcharges'].isna()].index)
l_index_charg

[161, 416, 1766, 2145, 3375, 3404, 4008, 5151]

In [33]:
df_churn_train.iloc[5151:5153]

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
5151,1341,1,0,1,1,0.0,0,3,2,1,1,1,1,1,0,0,0,4,56.05,NaN,0
5152,3749,0,0,0,0,48.0,1,0,2,0,1,1,1,1,1,0,0,4,78.90,3771.5,0


In [34]:
for i in l_index_charg:
    df_churn_train['totalcharges'][i] = 0.0

In [35]:
df_churn_train.isna().sum()

id                  0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [36]:
df_churn_train.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,4030,1,0,0,0,56.0,0,3,2,1,1,1,1,0,0,0,0,3,45.05,2560.10,0
1,6731,0,0,1,1,17.0,1,0,0,2,2,2,2,2,2,1,0,3,19.65,332.65,0
2,6479,1,0,1,0,60.0,1,1,1,0,1,1,1,1,1,0,1,4,110.80,6640.70,0
3,6861,1,0,0,0,37.0,1,1,1,0,1,1,0,1,1,2,1,2,101.90,3545.35,1
4,3266,0,0,1,1,29.0,1,0,1,0,0,0,0,0,0,2,1,1,70.75,1974.80,1


In [37]:
df_churn_train_trat = df_churn_train.copy()

In [75]:
df_churn_train_trat.columns

Index(['id', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [76]:
df_churn_train_trat.drop(columns=['seniorcitizen'], inplace=True)

In [84]:
df_churn_train_trat.drop(columns=['multiplelines'], inplace=True)

In [103]:
df_churn_train_trat.drop(columns=['paperlessbilling'], inplace=True)

In [115]:
df_churn_train_trat.drop(columns=['paymentmethod'], inplace=True)

In [120]:
df_churn_train_trat.drop(columns=['streamingtv'], inplace=True)

In [122]:
df_churn_train_trat.drop(columns=['streamingmovies'], inplace=True)

In [129]:
df_churn_train_trat.drop(columns=['phoneservice'], inplace=True)

#####tratar test

In [38]:
url = 'https://raw.githubusercontent.com/jjefferson2006/DSWP_JJ_altera/main/test.csv'
df_churn_test = pd.read_csv(url)
df_churn_test

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.00,445.3
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,NaN,99.00,5969.3
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15
3,6554,Female,0,Yes,Yes,61.0,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15
4,364,Female,0,No,No,47.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,20.55,945.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4897,Male,0,No,NaN,24.0,Yes,No,DSL,No,No,Yes,No,No,No,Month-to-month,Yes,Mailed check,49.70,1167.8
1405,6940,Male,0,No,No,35.0,Yes,No,Fiber optic,Yes,No,No,Yes,No,Yes,One year,Yes,Electronic check,89.20,3251.3
1406,804,Female,0,Yes,No,46.0,Yes,No,DSL,No,Yes,No,Yes,Yes,No,Two year,Yes,Credit card (automatic),64.20,3009.5
1407,1143,Male,1,Yes,Yes,11.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),75.20,775.3


In [39]:
#Normalizar as colunas
df_churn_test.columns = [col.lower() for col in df_churn_test.columns]
df_churn_test.head()

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,5027,Male,0,Yes,Yes,23.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),20.00,445.3
1,1733,Male,1,Yes,Yes,61.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,No,NaN,99.00,5969.3
2,5384,Male,0,No,No,36.0,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,84.75,3050.15
3,6554,Female,0,Yes,Yes,61.0,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),61.45,3751.15
4,364,Female,0,No,No,47.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,20.55,945.7


In [40]:
df_churn_test['dependents'].fillna('Yes', inplace=True)

In [41]:
df_churn_test['paymentmethod'].fillna('Mailed check', inplace=True)

In [42]:
df_churn_test['totalcharges'] = df_churn_test['totalcharges'].str.replace(',', '.')
df_churn_test['totalcharges'] = pd.to_numeric(df_churn_test['totalcharges'], errors= 'coerce')

In [43]:
df_churn_test[['totalcharges']].isna().sum()

totalcharges    3
dtype: int64

In [44]:
df_churn_test['tenure'].isna().sum()

118

In [ ]:
#atenção

In [45]:
df_churn_test['tenure_esti']=round(df_churn_test['totalcharges']/df_churn_test['monthlycharges'],2)

In [46]:
df_churn_test['tenure_rou']=round(df_churn_test['tenure_esti'])

In [47]:
l_tot_tes = list(df_churn_test[df_churn_test['tenure'].isna()].index)
l_tot_tes

[8,
 15,
 32,
 38,
 41,
 54,
 69,
 82,
 113,
 117,
 118,
 125,
 149,
 172,
 173,
 185,
 193,
 213,
 238,
 246,
 268,
 277,
 287,
 295,
 318,
 324,
 339,
 351,
 355,
 363,
 377,
 417,
 467,
 476,
 483,
 487,
 495,
 498,
 511,
 513,
 516,
 517,
 518,
 561,
 562,
 563,
 573,
 599,
 624,
 629,
 635,
 638,
 641,
 654,
 660,
 662,
 669,
 677,
 678,
 728,
 729,
 734,
 740,
 747,
 764,
 765,
 791,
 796,
 805,
 817,
 820,
 827,
 835,
 842,
 861,
 888,
 897,
 910,
 912,
 918,
 925,
 926,
 928,
 952,
 953,
 958,
 973,
 979,
 987,
 1009,
 1012,
 1022,
 1034,
 1045,
 1051,
 1069,
 1107,
 1159,
 1163,
 1169,
 1173,
 1189,
 1200,
 1213,
 1222,
 1224,
 1254,
 1276,
 1303,
 1310,
 1325,
 1344,
 1367,
 1368,
 1385,
 1387,
 1389,
 1394]

In [48]:
for i in l_tot_tes:
    df_churn_test['tenure'][i] = df_churn_test['tenure_rou'][i]


In [49]:
df_churn_test.drop(columns=['tenure_esti','tenure_rou'], inplace=True)

In [50]:
l_index_chargt = list(df_churn_test[df_churn_test['totalcharges'].isna()].index)
l_index_chargt

[313, 1063, 1297]

In [51]:
for j in l_index_chargt:
    df_churn_test['totalcharges'][j] = 0.0

In [52]:
df_churn_test.isna().sum()

id                  0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
dtype: int64

In [53]:
df_churn_test.gender.replace(['Female', 'Male'], [1, 0], inplace=True)
df_churn_test.partner.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_test.phoneservice.replace(['Yes', 'No'], [1, 0], inplace=True)
df_churn_test.paperlessbilling.replace(['Yes', 'No'], [1, 0], inplace=True)

In [54]:
df_churn_test.internetservice.replace(['Fiber optic', 'DSL', 'No'], [1,2,0], inplace=True)
df_churn_test.paymentmethod.replace(['Electronic check', 'Bank transfer (automatic)', 'Mailed check', 'Credit card (automatic)'], [1,2,3,4], inplace=True)
df_churn_test.contract.replace(['Two year', 'One year', 'Month-to-month'], [0,1,2], inplace=True)
df_churn_test.multiplelines.replace(['No', 'Yes', 'No phone service'], [0,1,3], inplace=True)
df_churn_test.onlinesecurity.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 
df_churn_test.onlinebackup.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.deviceprotection.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.techsupport.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.streamingtv.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True)
df_churn_test.streamingmovies.replace(['No', 'Yes', 'No internet service'], [0,1,2], inplace=True) 


In [55]:
df_churn_test.dependents.replace(['Yes', 'No'], [1, 0], inplace=True)

In [56]:
df_churn_test.isna().sum()

id                  0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
dtype: int64

In [57]:
df_churn_test

,id,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,5027,0,0,1,1,23.0,1,0,0,2,2,2,2,2,2,1,1,4,20.00,445.30
1,1733,0,1,1,1,61.0,1,1,1,0,1,0,0,1,1,1,0,3,99.00,5969.30
2,5384,0,0,0,0,36.0,1,1,1,0,0,0,0,0,1,2,1,1,84.75,3050.15
3,6554,1,0,1,1,61.0,0,3,2,0,1,1,1,1,1,0,1,2,61.45,3751.15
4,364,1,0,0,0,47.0,1,0,0,2,2,2,2,2,2,0,1,3,20.55,945.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4897,0,0,0,1,24.0,1,0,2,0,0,1,0,0,0,2,1,3,49.70,1167.80
1405,6940,0,0,0,0,35.0,1,0,1,1,0,0,1,0,1,1,1,1,89.20,3251.30
1406,804,1,0,1,0,46.0,1,0,2,0,1,0,1,1,0,0,1,4,64.20,3009.50
1407,1143,0,1,1,1,11.0,1,1,1,0,0,0,0,0,0,2,1,4,75.20,775.30


In [58]:
df_churn_train.columns

Index(['id', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
       'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [ ]:
'tenure', 'monthlycharges', 'totalcharges'

In [59]:
df_churn_test_trat = df_churn_test.copy()

In [77]:
df_churn_test_trat.drop(columns=['seniorcitizen'], inplace=True)

In [85]:
df_churn_test_trat.drop(columns=['multiplelines'], inplace=True)

In [102]:
df_churn_test_trat.drop(columns=['paperlessbilling'], inplace=True)

In [114]:
df_churn_test_trat.drop(columns=['paymentmethod'], inplace=True)

In [118]:
df_churn_test_trat.drop(columns=['streamingtv'], inplace=True)

In [125]:
df_churn_test_trat.drop(columns=['streamingmovies'], inplace=True)

In [128]:
df_churn_test_trat.drop(columns=['phoneservice'], inplace=True)

###automl

In [85]:
'''clf = setup(data = df_churn_train,
            target = 'churn',
            #numeric_features = ['tenure', 'monthlycharges', 'totalcharges'] 
            #numeric_imputation = 'mean'
            categorical_features = ['gender', 'seniorcitizen', 'partner', 'dependents',
            'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
            'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
            'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod'], # lista das variáveis categóricas
            #normalize =True,
            normalize_method = 'minmax',
            bin_numeric_features = ['tenure', 'monthlycharges', 'totalcharges'],
            transformation = True,
            #pca = True,
            ignore_low_variance = True,
            ignore_features = ['id'],
            fix_imbalance = True,
            pca = True, 
            silent = False)

,Description,Value
0,session_id,8262
1,Target,churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(5634, 21)"
5,Missing Values,True
6,Numeric Features,3
7,Categorical Features,16
8,Ordinal Features,False
9,High Cardinality Features,False


In [130]:
clf = setup(data = df_churn_train_trat,
            target = 'churn',
            #numeric_features = ['tenure', 'monthlycharges', 'totalcharges'] 
            #numeric_imputation = 'mean'
            categorical_features = ['gender', 'partner', 'dependents',
            'internetservice', 'onlinesecurity',
            'onlinebackup', 'deviceprotection', 'techsupport', 'contract'], # lista das variáveis categóricas
            silent = False)

,Description,Value
0,session_id,2149
1,Target,churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(5634, 14)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [131]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8017,0.8492,0.7155,0.7930,0.7937,0.4606,0.4671,0.512
lr,Logistic Regression,0.7984,0.8409,0.7236,0.7924,0.7936,0.4658,0.4690,0.082
catboost,CatBoost Classifier,0.7984,0.8465,0.7120,0.7898,0.7904,0.4524,0.4590,3.434
rf,Random Forest Classifier,0.7969,0.8354,0.7095,0.7876,0.7888,0.4478,0.4539,0.718
ridge,Ridge Classifier,0.7946,0.0000,0.7067,0.7848,0.7864,0.4413,0.4470,0.018
ada,Ada Boost Classifier,0.7941,0.8421,0.7213,0.7888,0.7900,0.4580,0.4605,0.213
lda,Linear Discriminant Analysis,0.7936,0.8391,0.7183,0.7873,0.7889,0.4538,0.4565,0.028
xgboost,Extreme Gradient Boosting,0.7865,0.8297,0.7042,0.7784,0.7802,0.4292,0.4329,0.691
lightgbm,Light Gradient Boosting Machine,0.7854,0.8349,0.6999,0.7761,0.7782,0.4222,0.4265,0.112
et,Extra Trees Classifier,0.7827,0.8191,0.7034,0.7755,0.7775,0.4238,0.4265,0.621


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2149, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [108]:
lda = create_model('lda')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8253,0.8772,0.7595,0.8204,0.8221,0.5370,0.5385
1,0.8025,0.8307,0.7106,0.7921,0.7937,0.4543,0.4608
2,0.7975,0.8257,0.7122,0.7880,0.7903,0.4515,0.4558
3,0.7563,0.8038,0.6611,0.7443,0.7486,0.3408,0.3434
4,0.7817,0.8349,0.6966,0.7725,0.7757,0.4132,0.4157
5,0.8046,0.8505,0.7576,0.8077,0.8060,0.5076,0.5078
6,0.8452,0.8802,0.7671,0.8395,0.8386,0.5743,0.5818
7,0.7944,0.8237,0.7234,0.7893,0.7914,0.4594,0.4602
8,0.8096,0.8323,0.7490,0.8069,0.8081,0.5056,0.5059
9,0.7919,0.8176,0.7005,0.7810,0.7835,0.4296,0.4347


In [109]:
tuned_lda = tune_model(lda)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8253,0.8772,0.7595,0.8204,0.8221,0.5370,0.5385
1,0.8025,0.8307,0.7106,0.7921,0.7937,0.4543,0.4608
2,0.7975,0.8257,0.7122,0.7880,0.7903,0.4515,0.4558
3,0.7563,0.8038,0.6611,0.7443,0.7486,0.3408,0.3434
4,0.7817,0.8349,0.6966,0.7725,0.7757,0.4132,0.4157
5,0.8046,0.8505,0.7576,0.8077,0.8060,0.5076,0.5078
6,0.8452,0.8802,0.7671,0.8395,0.8386,0.5743,0.5818
7,0.7944,0.8237,0.7234,0.7893,0.7914,0.4594,0.4602
8,0.8096,0.8323,0.7490,0.8069,0.8081,0.5056,0.5059
9,0.7919,0.8176,0.7005,0.7810,0.7835,0.4296,0.4347


In [94]:
catboost = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8101,0.8715,0.7228,0.8010,0.8028,0.4766,0.4818
1,0.7848,0.8381,0.7005,0.7769,0.7799,0.4181,0.4197
2,0.8000,0.8400,0.7013,0.7885,0.7903,0.4370,0.4441
3,0.8223,0.8558,0.7386,0.8144,0.8157,0.5091,0.5142
4,0.8376,0.8735,0.7865,0.8366,0.8370,0.5767,0.5768
5,0.7843,0.8409,0.6815,0.7709,0.7738,0.3938,0.4002
6,0.8249,0.8644,0.7403,0.8170,0.8180,0.5145,0.5202
7,0.8223,0.8608,0.7543,0.8175,0.8193,0.5250,0.5262
8,0.8096,0.8572,0.7363,0.8039,0.8060,0.4894,0.4908
9,0.8249,0.8636,0.7278,0.8163,0.8147,0.5011,0.5125


In [95]:
tuned_catboost = tune_model(catboost)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8228,0.8719,0.7283,0.8141,0.8131,0.4999,0.5102
1,0.7899,0.8452,0.7039,0.7813,0.7843,0.4280,0.4303
2,0.8076,0.8428,0.7033,0.7962,0.7959,0.4490,0.4600
3,0.8173,0.8538,0.7320,0.8089,0.8105,0.4951,0.5000
4,0.8173,0.8707,0.7508,0.8131,0.8148,0.5146,0.5153
5,0.7944,0.8441,0.6977,0.7829,0.7854,0.4262,0.4320
6,0.8223,0.8706,0.7355,0.8140,0.8149,0.5058,0.5120
7,0.8147,0.8659,0.7303,0.8064,0.8082,0.4898,0.4942
8,0.8046,0.8609,0.7266,0.7977,0.8001,0.4724,0.4743
9,0.8274,0.8719,0.7358,0.8192,0.8186,0.5134,0.5225


In [62]:
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8152,0.8615,0.7394,0.8094,0.8116,0.4965,0.4979
1,0.8278,0.8367,0.7251,0.8190,0.8170,0.4989,0.5115
2,0.8304,0.8631,0.7366,0.8220,0.8219,0.5150,0.5236
3,0.8376,0.8747,0.7856,0.8376,0.8376,0.5712,0.5712
4,0.7919,0.8304,0.6890,0.7796,0.7828,0.4075,0.4128
5,0.8376,0.8771,0.7559,0.8308,0.8319,0.5441,0.5486
6,0.8147,0.8259,0.7472,0.8118,0.8131,0.5026,0.5029
7,0.8071,0.8736,0.7276,0.8005,0.8030,0.4736,0.4752
8,0.8122,0.8278,0.7180,0.8023,0.8041,0.4695,0.4755
9,0.8147,0.8528,0.7457,0.8109,0.8125,0.5027,0.5033


In [63]:
tuned_lr = tune_model(lr)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8278,0.8538,0.7446,0.8204,0.8220,0.5196,0.5239
1,0.8278,0.8368,0.7251,0.8190,0.8170,0.4989,0.5115
2,0.8304,0.8632,0.7366,0.8220,0.8219,0.5150,0.5236
3,0.8376,0.8747,0.7856,0.8376,0.8376,0.5712,0.5712
4,0.7919,0.8304,0.6890,0.7796,0.7828,0.4075,0.4128
5,0.8376,0.8769,0.7559,0.8308,0.8319,0.5441,0.5486
6,0.8147,0.8260,0.7472,0.8118,0.8131,0.5026,0.5029
7,0.8325,0.8826,0.7511,0.8254,0.8268,0.5334,0.5378
8,0.8122,0.8277,0.7180,0.8023,0.8041,0.4695,0.4755
9,0.8173,0.8526,0.7409,0.8112,0.8133,0.5013,0.5030


In [132]:
gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8228,0.8779,0.7325,0.8147,0.8134,0.5075,0.5177
1,0.8000,0.8481,0.7020,0.7886,0.7889,0.4424,0.4519
2,0.7899,0.8321,0.6738,0.7764,0.7720,0.3952,0.4141
3,0.8147,0.8561,0.7510,0.8103,0.8120,0.5159,0.5169
4,0.7792,0.8565,0.6999,0.7715,0.7744,0.4159,0.4175
5,0.8122,0.8318,0.7225,0.8029,0.8031,0.4824,0.4905
6,0.8020,0.8437,0.7215,0.7936,0.7958,0.4681,0.4717
7,0.7970,0.8309,0.7150,0.7882,0.7906,0.4545,0.4579
8,0.7868,0.8478,0.7141,0.7810,0.7833,0.4413,0.4423
9,0.8122,0.8669,0.7225,0.8029,0.8031,0.4824,0.4905


In [133]:
tuned_gbc = tune_model(gbc)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8101,0.8751,0.7269,0.8015,0.8030,0.4841,0.4892
1,0.7924,0.8419,0.6939,0.7802,0.7814,0.4231,0.4315
2,0.7949,0.8244,0.6832,0.7826,0.7786,0.4137,0.4313
3,0.8249,0.8652,0.7610,0.8201,0.8217,0.5396,0.5411
4,0.7766,0.8569,0.6982,0.7694,0.7722,0.4111,0.4124
5,0.8046,0.8342,0.7113,0.7942,0.7947,0.4596,0.4681
6,0.7944,0.8327,0.7103,0.7851,0.7876,0.4459,0.4497
7,0.7868,0.8275,0.7021,0.7773,0.7801,0.4272,0.4305
8,0.7944,0.8475,0.7252,0.7894,0.7914,0.4629,0.4637
9,0.8096,0.8555,0.7207,0.8002,0.8009,0.4770,0.4845


In [66]:
ada = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8051,0.8345,0.7205,0.7966,0.7991,0.4658,0.4691
1,0.8152,0.8771,0.7556,0.8135,0.8143,0.5161,0.5162
2,0.8000,0.8632,0.7359,0.7982,0.7990,0.4763,0.4764
3,0.8122,0.8697,0.7349,0.8053,0.8075,0.4913,0.4936
4,0.8122,0.8418,0.7349,0.8053,0.8075,0.4913,0.4936
5,0.7690,0.8003,0.6649,0.7548,0.7589,0.3554,0.3602
6,0.8020,0.8599,0.7186,0.7938,0.7963,0.4603,0.4631
7,0.7970,0.8411,0.7057,0.7868,0.7894,0.4390,0.4434
8,0.8020,0.8622,0.7060,0.7911,0.7929,0.4456,0.4522
9,0.8401,0.8776,0.7851,0.8378,0.8388,0.5793,0.5797


In [67]:
tuned_ada = tune_model(ada)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8152,0.8324,0.7336,0.8074,0.8095,0.4936,0.4970
1,0.8278,0.8755,0.7767,0.8278,0.8278,0.5535,0.5535
2,0.7975,0.8648,0.7248,0.7928,0.7947,0.4611,0.4618
3,0.8274,0.8763,0.7609,0.8228,0.8245,0.5386,0.5398
4,0.7970,0.8418,0.7151,0.7893,0.7919,0.4500,0.4521
5,0.7868,0.8027,0.7020,0.7785,0.7815,0.4225,0.4245
6,0.8147,0.8674,0.7397,0.8084,0.8105,0.4998,0.5018
7,0.7995,0.8440,0.7012,0.7880,0.7898,0.4366,0.4437
8,0.8096,0.8625,0.7206,0.8003,0.8022,0.4723,0.4775
9,0.8350,0.8761,0.7786,0.8326,0.8337,0.5660,0.5663


In [56]:
lr = create_model('lr') 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7873,0.8034,0.7053,0.7787,0.7815,0.4313,0.4338
1,0.8203,0.8639,0.7547,0.8153,0.8170,0.5267,0.5281
2,0.7899,0.8442,0.6981,0.7787,0.7811,0.4254,0.4309
3,0.7868,0.8058,0.7304,0.7881,0.7874,0.4580,0.4580
4,0.7919,0.8635,0.6974,0.7804,0.7826,0.4259,0.4320
5,0.8223,0.8608,0.7455,0.8152,0.8167,0.5193,0.5233
6,0.7970,0.8307,0.7100,0.7873,0.7897,0.4471,0.4514
7,0.8223,0.8800,0.7533,0.8165,0.8182,0.5286,0.5310
8,0.8122,0.8634,0.7434,0.8065,0.8085,0.5048,0.5064
9,0.8071,0.8330,0.7131,0.7969,0.7969,0.4649,0.4742


In [57]:
tuned_lr = tune_model(lr)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7873,0.8033,0.7053,0.7787,0.7815,0.4313,0.4338
1,0.8278,0.8635,0.7629,0.8228,0.8244,0.5453,0.5471
2,0.7899,0.8443,0.6981,0.7787,0.7811,0.4254,0.4309
3,0.8020,0.8076,0.7407,0.7997,0.8008,0.4874,0.4876
4,0.8071,0.8662,0.7169,0.7973,0.7985,0.4679,0.4746
5,0.8198,0.8560,0.7407,0.8123,0.8137,0.5109,0.5153
6,0.7970,0.8306,0.7100,0.7873,0.7897,0.4471,0.4514
7,0.8350,0.8838,0.7739,0.8306,0.8321,0.5663,0.5679
8,0.8147,0.8654,0.7421,0.8081,0.8100,0.5069,0.5095
9,0.8147,0.8328,0.7332,0.8065,0.8078,0.4975,0.5027


In [138]:
predic = predict_model(tuned_gbc, data = df_churn_test_trat)
predic

,id,gender,partner,dependents,tenure,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,contract,monthlycharges,totalcharges,Label,Score
0,5027,0,1,1,23.0,0,2,2,2,2,1,20.00,445.30,0,0.9759
1,1733,0,1,1,61.0,1,0,1,0,0,1,99.00,5969.30,0,0.7031
2,5384,0,0,0,36.0,1,0,0,0,0,2,84.75,3050.15,1,0.5458
3,6554,1,1,1,61.0,2,0,1,1,1,0,61.45,3751.15,0,0.9795
4,364,1,0,0,47.0,0,2,2,2,2,0,20.55,945.70,0,0.9951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4897,0,0,1,24.0,2,0,0,1,0,2,49.70,1167.80,0,0.7820
1405,6940,0,0,0,35.0,1,1,0,0,1,1,89.20,3251.30,0,0.9284
1406,804,1,1,0,46.0,2,0,1,0,1,0,64.20,3009.50,0,0.9712
1407,1143,0,1,1,11.0,1,0,0,0,0,2,75.20,775.30,1,0.5564


In [139]:
predic['Score'].mean()

0.8114442867281765

### gbc = 0.8136 - k = 0.7826
###rid =0.80 - k=0.77
###cat = 0.8002 k =0.7897
###gbc = 0.811 - k = 0.7698
###cat =0.8276 - k = 0.77
###lda = 0.8192 - k =0.7826

In [140]:
pred_val = predic['Label']

In [67]:
from google.colab import files

In [141]:
ids = df_churn_test['id']
output = pd.DataFrame({'id': ids, 'Churn': pred_val})
output.to_csv('submission25anagbc.csv', index=False)
files.download('submission25anagbc.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>